In [14]:
import numpy as np
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
from sklearn import metrics
from catboost import Pool,CatBoostRegressor,CatBoostClassifier
import random
import time
import datetime
patch="D:\Python\\Google"
def run_lgb(train_x, train_y, val_x, val_y,val_id):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "learning_rate" : 0.02,
        'n_estimators': 50,
         'num_iterations': 50,
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    
    pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].mean().reset_index()
#    return model
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))
def run_cat(train_x, train_y, val_x, val_y,val_id):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    
    pred_val = model.predict(val_x)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].mean().reset_index()
    return model
#    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))

def run_cat_bin(train_x, train_y, val_x, val_y,val_id):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostClassifier(iterations=80, learning_rate=0.05,loss_function='CrossEntropy', eval_metric='AUC')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    pred_train=model.predict(train_x, prediction_type="Probability")[:,1]
    pred_val = model.predict(val_x, prediction_type="Probability")[:,1]
#    return val_pred_df
    return  pred_train,pred_val

def run_lgb_bin(train_x, train_y, val_x, val_y,val_id):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        "learning_rate" : 0.02,
        'n_estimators': 50,
         'num_iterations': 50,
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    
    pred_train = model.predict(train_x, num_iteration=model.best_iteration)
    pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    
#    return model
    return  pred_train,pred_val

    
def multimodel(train_x, train_y, val_x, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "learning_rate" : 0.01,
        'n_estimators': 50,
         'num_iterations': 70,
    }
    i=random.randint(0,99999)
    train1=pd.concat([train_x.reset_index(drop=True),pd.DataFrame(train_y,columns=['bg'])],axis=1)
    train_x_bad=train1[train1['bg']>0].drop(['bg'],axis=1)
    train_y_bad=train1[train1['bg']>0]['bg'].values
    train_x_good=train1[train1['bg']==0].drop(['bg'],axis=1).sample(len(train_x_bad)*6,random_state=i)
    train_y_good=train1[train1['bg']==0]['bg'].sample(len(train_x_bad)*6,random_state=i).values
    
    val1=pd.concat([val_x.reset_index(drop=True),pd.DataFrame(val_y,columns=['bg'])],axis=1)
    val_x_bad=val1[val1['bg']>0].drop(['bg'],axis=1)
    val_y_bad=val1[val1['bg']>0]['bg'].values
    val_x_good=val1[val1['bg']==0].drop(['bg'],axis=1).sample(len(val_x_bad)*6,random_state=i)
    val_y_good=val1[val1['bg']==0]['bg'].sample(len(val_x_bad)*6,random_state=i).values
    
    train_x,train_y=pd.concat([train_x_bad,train_x_good]),np.concatenate([train_y_bad,train_y_good])
    val_x,val_y=pd.concat([val_x_bad,val_x_good]),np.concatenate([val_y_bad,val_y_good])
#     pool=Pool(train_x,train_y)
#     val_pool=Pool(val_x,val_y)
#     model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
#     model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    return model

def run_multimodel(train_x, train_y, val_x, val_y,val_id):
    result_val=[]
    for i in range(0,10):
        model=multimodel(train_x, train_y, val_x, val_y)
        result_val.append(model.predict(val_x, num_iteration=model.best_iteration))
    pred_val = sum(result_val)/float(len(result_val))
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
#    return val_pred_df
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))

In [15]:
train=pd.read_pickle(os.path.join(patch,'train_v2_clean.pkl'))
#train_spare=train.copy()
#np.concatenate([train_y_bad,train_y_good.values])


In [16]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

#train=grouping_month(train)
train["totals.totalTransactionRevenue"].fillna(0, inplace=True)
train["visitStartTime"] = pd.to_datetime(train["visitStartTime"],unit='s')

train['day']=train['visitStartTime'].dt.day.astype('uint8')
train['weekday']=train['visitStartTime'].apply(lambda x:x.date().weekday())
train['minute']=train['visitStartTime'].dt.minute.astype('uint8')
train['hour']=train['visitStartTime'].dt.hour.astype('uint8')

# label encode the categorical variables and convert the numerical variables to float
#ar=df['geoNetwork.networkDomain'].value_counts()[df['geoNetwork.networkDomain'].value_counts()<=4].index
#train['geoNetwork.networkDomain']=['some unfamous domain' if x in ar else x for x in df['geoNetwork.networkDomain'].values]
#del ar
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect',
           'customDimensions_value','weekday']
for col in cat_cols:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
#train=pd.get_dummies(train,columns=cat_cols,prefix=cat_cols,prefix_sep='_')

num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits',
            'totals.transactions','totals.sessionQualityDim','totals.timeOnSite','day','minute','hour']

# num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits',
#            'totals.transactions','totals.sessionQualityDim','totals.timeOnSite','day','minute','hour',
#             'totals.totalTransactionRevenue_month_sum',
#            'totals.totalTransactionRevenue_month_count', 'totals.hits_month_sum',
#            'totals.hits_month_count', 'totals.hits_month_median',
#            'totals.pageviews_month_sum', 'totals.pageviews_month_mean',
#            'totals.pageviews_month_median', 'totals.bounces_month_sum',
#            'totals.bounces_month_mean', 'totals.bounces_month_median',
#            'totals.newVisits_month_sum', 'totals.newVisits_month_mean',
#            'totals.newVisits_month_median']    
for col in num_cols:
    train[col] = train[col].astype(float)
    
train['totals.totalTransactionRevenue'] = train['totals.totalTransactionRevenue'].astype(float)
train['totals.totalTransactionCount'] = np.where(train['totals.totalTransactionRevenue'].values>0,1,0)
train['device.isMobile']=train['device.isMobile'].astype(int)

# Split the train dataset into development and valid based on time 
visitid=train['fullVisitorId']
train=train.drop(['date','visitId','totals.transactionRevenue'],axis=1)

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect
customDimensions_value
weekday


In [32]:
#np.datetime64('2018-03-01')-pd.to_datetime(df['last_visit'],unit='s')
#np.datetime64('2018-03-01')-pd.to_datetime(df['last_visit'],unit='s')
#pd.to_datetime(df['last_visit'],unit='s').apply(lambda x:x-np.datetime64('2018-03-01'))

#pd.to_datetime(df['last_visit'],unit='s')
#pd.to_datetime(df['last_visit'],unit='s')

In [85]:
result=pd.DataFrame(columns=['iteration','learning_rate','mean'])
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2016,11,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2016,12,1))&(train['visitStartTime']<datetime.date(2017,2,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2017,8,1))&(train['visitStartTime']<datetime.date(2017,11,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2017,12,1))&(train['visitStartTime']<datetime.date(2018,2,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[train_x_index]
train_x=grouping_all(train.loc[train_x_index])
#train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
val_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[val_x_index],df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[val_x_index]
val_x=grouping_all(train.loc[val_x_index])
#val_x=grouping_date(val_x.loc[val_x_index],2017,11,1)
val_x=val_x.drop(['visitStartTime','fullVisitorId'],axis=1)
#result=pd.DataFrame(columns=['estimator','mean'])
for itr in [50,80,110,140,200,250,350]:
    for rate in [0.01,0.02,0.03,0.05,0.1]:
    # for estim in [20,40,50,60,70,90,110,130,170]:
    #     average=[]
        result.loc[len(result)]=[itr,rate,run_cat(train_x,train_y,val_x,val_y,val_id,itr,rate)]
#a=run_lgb(train_x,train_y,val_x,val_y,val_id)    

In [25]:
#итоговая фигня
import os
patch="D:\Python\\Google"
train=pd.read_pickle(os.path.join(patch,'train_v2_clean.pkl'))
test=pd.read_pickle(os.path.join(patch,'test_v2_clean.pkl'))
from sklearn.preprocessing import LabelEncoder
train["totals.totalTransactionRevenue"].fillna(0, inplace=True)


train["visitStartTime"] = pd.to_datetime(train["visitStartTime"],unit='s')
test["visitStartTime"] = pd.to_datetime(test["visitStartTime"],unit='s')

train['day']=train['visitStartTime'].dt.day.astype('uint8')
train['weekday']=train['visitStartTime'].apply(lambda x:x.date().weekday())
train['minute']=train['visitStartTime'].dt.minute.astype('uint8')
train['hour']=train['visitStartTime'].dt.hour.astype('uint8')

test['day']=test['visitStartTime'].dt.day.astype('uint8')
test['weekday']=test['visitStartTime'].apply(lambda x:x.date().weekday())
test['minute']=test['visitStartTime'].dt.minute.astype('uint8')
test['hour']=test['visitStartTime'].dt.hour.astype('uint8')

train=train.drop(['date','visitId','totals.transactionRevenue'],axis=1)
test=test.drop(['date','visitId','totals.transactionRevenue'],axis=1)

# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect','customDimensions_value','weekday']
for col in cat_cols:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits',
           'totals.transactions','totals.sessionQualityDim','totals.timeOnSite','day','minute','hour']    
for col in num_cols:
    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)
    
train['totals.totalTransactionRevenue'] = train['totals.totalTransactionRevenue'].astype(float)
train['device.isMobile']=train['device.isMobile'].astype(int)
test['device.isMobile']=test['device.isMobile'].astype(int)

# Split the train dataset into development and valid based on time 
def grouping_all(df):
    aggs={
        'totals.totalTransactionRevenue': ['sum','count'],
        'totals.hits': ['sum','count', 'median'],
        'totals.pageviews': ['sum','mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
    temp=df.groupby(['fullVisitorId'],as_index=False).agg({**aggs})
    col=["_all_".join(x) for x in temp.columns.ravel()[1:]]
    col.insert(0,'fullVisitorId')
    temp.columns=col
    #temp.columns=['fullVisitorId','Month','transaction_month_count']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    df.fillna(0, inplace=True)
    return df

test_id = test["fullVisitorId"].values
test=grouping_all(test)
test=test.drop(['visitStartTime','fullVisitorId'],axis=1)

def run_lgb(train_X, train_y, val_X, val_y, test_X):
    pool=Pool(train_X,train_y)
    val_pool=Pool(val_X,val_y)
    model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    
    pred_test_y = model.predict(test_X)
    return pred_test_y, model
# Training the model #
def run_cat(train_x, train_y, val_x, val_y,test):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    
    pred_test = model.predict(test)
#    return val_pred_df
    return  pred_test
import time
import datetime
from catboost import Pool,CatBoostRegressor
train_x_index=train[(train['visitStartTime']>=datetime.date(2017,6,1))&(train['visitStartTime']<datetime.date(2017,11,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,12,1))&(train['visitStartTime']<datetime.date(2018,2,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2016,11,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2016,12,1))&(train['visitStartTime']<datetime.date(2017,2,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=grouping_all(train_x.loc[train_x_index])
train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
val_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[val_x_index],df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_x=grouping_all(val_x.loc[val_x_index])
val_x=train.drop(['visitStartTime','fullVisitorId'],axis=1)

pred_test=run_cat(train_x,train_y,val_x,val_y,test)
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_test[pred_test<0] = 0
sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.to_csv("predict.csv", index=False)

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect
customDimensions_value
weekday


In [20]:
#train["date"]=pd.to_datetime(train["date"], format='%Y%m%d')
#train['Month']=train["date"].dt.to_period("M")
def grouping_month(df):
    df["date"]=pd.to_datetime(df["date"], format='%Y%m%d')
    df['Month']=df["date"].dt.to_period("M")
    df['totals.hits']=df['totals.hits'].astype(int)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.newVisits']=df['totals.newVisits'].astype(int)
    df['totals.pageviews'].fillna(0, inplace=True)
    df['totals.pageviews']=df['totals.pageviews'].astype(int)
    df['totals.bounces'].fillna(0, inplace=True)
    df['totals.bounces']=df['totals.bounces'].astype(int)
    df['totals.totalTransactionRevenue'].fillna(0, inplace=True)
    df['totals.totalTransactionRevenue']=df['totals.totalTransactionRevenue'].astype(float)
    aggs={
        'totals.totalTransactionRevenue': ['sum','count'],
        'totals.hits': ['sum','count', 'median'],
        'totals.pageviews': ['sum','mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
    temp=df.groupby(['fullVisitorId','Month'],as_index=False).agg({**aggs})
    col=["_month_".join(x) for x in temp.columns.ravel()[2:]]
    col.insert(0,'Month')
    col.insert(0,'fullVisitorId')
    temp.columns=col
    #temp.columns=['fullVisitorId','Month','transaction_month_count']
    df=pd.merge(df,temp,on=['fullVisitorId','Month'],how='left')
    return df
def grouping_all(df):
    aggs={
        'totals.totalTransactionRevenue': ['sum'],
        'totals.totalTransactionCount': ['sum','mean'],
        'totals.hits': ['sum','count', 'median'],
        'totals.pageviews': ['sum','mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
    temp=df.groupby(['fullVisitorId'],as_index=False).agg({**aggs})
    col=["_all_".join(x) for x in temp.columns.ravel()[1:]]
    col.insert(0,'fullVisitorId')
    temp.columns=col
    #temp.columns=['fullVisitorId','Month','transaction_month_count']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    df.fillna(0, inplace=True)
    return df
def grouping_date(df,y,m,d):
    temp=df.groupby('fullVisitorId',as_index=False)['visitStartTime'].min()
    temp.columns=['fullVisitorId','last_visit']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    temp=df[df['totals.totalTransactionRevenue']>0].groupby('fullVisitorId',as_index=False)['visitStartTime'].min()
    temp.columns=['fullVisitorId','last_visit_money']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    dat_cols=['visitStartTime','last_visit','last_visit_money']
    for col in dat_cols:
        df[col] = pd.to_datetime(df[col],unit='s')
    df['last_visit']=df['last_visit']-datetime.datetime(y,m,d)
    df['last_visit']=df['last_visit'].dt.days
    df['last_visit_money']=df['last_visit_money']-datetime.datetime(y,m,d)
    df['last_visit_money']=df['last_visit_money'].dt.days
    return df
    

In [21]:
# здесь все тестим
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2016,11,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2016,12,1))&(train['visitStartTime']<datetime.date(2017,2,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2017,8,1))&(train['visitStartTime']<datetime.date(2017,11,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2017,12,1))&(train['visitStartTime']<datetime.date(2018,2,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[train_x_index]
train_x=grouping_all(train.loc[train_x_index])
train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
val_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[val_x_index],df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[val_x_index]
val_x=grouping_all(train.loc[val_x_index])
val_x=grouping_date(val_x.loc[val_x_index],2017,11,1)
val_x=val_x.drop(['visitStartTime','fullVisitorId'],axis=1)

run_lgb(train_x,train_y,val_x,val_y,val_id)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 16 rounds.
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.847564


0.08532446062635884

In [7]:
# соответсвующие размеры, ранняя выборка 0.13582451736796636
#максимальный размер выборкаи 0.1314093794058963
# соответсвующие размеры, позняя выборка 0.13826095451093567
# соответсвующие размеры, одинаковая сезонность 0.12447279363543821
#baseline 0.11302635053856372
#Feature_important_lgb
# import matplotlib.pyplot as plt
# lgb.plot_importance(model)
# plt.show()

#Feature_important_cat
pd.DataFrame(list(zip(model.feature_importances_,train_x.columns.values)))
0.08526920794172219
0.08536053628414299

,0,1
0,1.036890,channelGrouping
1,0.029963,visitNumber
2,0.000000,device.browser
3,0.124776,device.deviceCategory
4,0.000000,device.isMobile
5,4.957348,device.operatingSystem
6,0.000000,geoNetwork.city
7,0.253637,geoNetwork.continent
8,0.000000,geoNetwork.country
9,0.000000,geoNetwork.metro


In [67]:
# здесь тестим с бинарным предсказанием
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,10,1))&(train['visitStartTime']<datetime.date(2017,3,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,3,1))&(train['visitStartTime']<datetime.date(2017,5,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2017,10,1))&(train['visitStartTime']<datetime.date(2018,3,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2018,3,1))&(train['visitStartTime']<datetime.date(2018,5,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[train_x_index]
train_x=grouping_all(train.loc[train_x_index])
train_x=grouping_date(train_x.loc[train_x_index],2017,3,1)
train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
val_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[val_x_index],df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[val_x_index]
val_x=grouping_all(train.loc[val_x_index])
val_x=grouping_date(val_x.loc[val_x_index],2018,3,1)
val_x=val_x.drop(['visitStartTime','fullVisitorId'],axis=1)

train_x['prob'],val_x['prob']=run_lgb_bin(train_x,np.where(train_y>0,1,0),val_x,np.where(val_y>0,1,0),val_id)

run_lgb(train_x,train_y,val_x,val_y,val_id)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 16 rounds.
Early stopping, best iteration is:
[3]	valid_0's auc: 0.502173
Training until validation scores don't improve for 16 rounds.
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.888331


0.10427778059466349

In [8]:
train

,channelGrouping,fullVisitorId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,...,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions_value,day,weekday,minute,hour
0,4,3162355547410993243,1.0,2017-10-17 00:00:50,43,0,0,20,955,4,...,1,3366,5,3196,99,2,17.0,1,0.0,0.0
1,6,8934116514970143966,6.0,2017-10-16 17:51:47,35,0,0,3,220,2,...,1,2746,6,300,301,3,16.0,0,51.0,17.0
2,2,7992466427990357681,1.0,2017-10-17 00:53:33,35,1,1,1,955,2,...,0,2746,0,3196,0,3,17.0,1,53.0,0.0
3,4,9075655783635761930,1.0,2017-10-16 16:04:11,35,0,0,20,955,3,...,1,7,5,3196,99,2,16.0,0,4.0,16.0
4,4,6960673291025684308,1.0,2017-10-16 21:49:12,35,0,0,20,955,2,...,1,7,5,3196,99,1,16.0,0,49.0,21.0
5,6,0166277907528479249,1.0,2017-10-16 23:31:41,35,0,0,7,749,2,...,1,2746,0,2254,0,3,16.0,0,31.0,23.0
6,6,8349655975937271469,1.0,2017-10-16 11:14:38,35,0,0,7,458,4,...,1,2746,6,249,301,2,16.0,0,14.0,11.0
7,4,1332629902468998662,1.0,2017-10-17 02:10:08,35,0,0,20,955,4,...,1,7,5,3196,99,5,17.0,1,10.0,2.0
8,4,632878546807742341,1.0,2017-10-17 02:31:56,35,0,0,7,515,2,...,1,7,5,3196,99,1,17.0,1,31.0,2.0
9,4,1259490915281096752,2.0,2017-10-16 14:45:59,72,1,1,23,955,4,...,0,7,5,3196,99,2,16.0,0,45.0,14.0
